In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt



In [18]:
df = pd.read_csv('data/dashboard_data.csv')
# df.iloc[: , 2:]
df = df.loc[: , ~df.columns.str.contains('Unnamed')]

In [19]:
mean_df = df.groupby('name').mean()
mean_df = mean_df.drop(columns=['year'])
mean_df

,impervious_fraction,max_3day_rain_mm,peak_30min_intensity_mm,sar_flood_freq_pct,potential_ha,static_water_ha,static_weed_ha,elevation,slope,flow_accumulation_km2,csr_ratio,log_flow,urban_stress,urban_surge,water_retention_ratio,weed_clogging_ratio,biological_clogging,rain_intensity_impact
name,,,,,,,,,,,,,,,,,,
Abbigere Lake,0.326674,81.433331,7.368333,16.895198,9.899677,1.206915,4.293971,883.016434,3.153620,5.506420,0.555661,1.872789,1.798803,2.321469,0.121792,0.433311,0.779180,2.321469
Agara Lake,0.023121,81.659998,7.086667,5.764079,26.724699,18.592974,4.461419,877.305438,1.844719,0.008336,0.000312,0.008301,0.000193,0.164046,0.695462,0.166877,0.193433,0.164046
Agrahara Lake,0.539475,84.547498,6.272500,13.790324,4.266708,2.033760,0.299592,899.184482,6.437909,2.798880,0.654447,1.334706,1.509925,3.296539,0.475543,0.070052,0.127848,3.296539
Akshaynagar Lake,0.945938,86.146664,6.710833,4.888746,1.187524,0.404341,0.467331,901.448762,2.535673,0.158408,0.132279,0.147046,0.149844,6.349636,0.337647,0.390248,0.530051,6.349636
Allalasandra Lake,0.184137,81.433331,7.368333,3.631073,13.948385,12.030289,1.809916,903.357495,1.944574,0.008330,0.000597,0.008295,0.001534,1.362220,0.861868,0.129665,0.130678,1.362220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yediyur Lake,0.679374,82.415831,7.633333,4.153899,3.884062,3.058791,0.696718,908.555966,6.183740,0.091688,0.023546,0.087725,0.062290,5.193787,0.785501,0.178918,0.185026,5.193787
Yelachenahalli Kere,0.908081,84.596826,7.038962,6.048937,1.549492,0.985786,0.225535,889.097619,3.799281,0.550203,0.352809,0.438386,0.499628,6.395668,0.632120,0.144621,0.184665,6.395668
Yelahanka Lake,0.000000,84.315748,7.978333,0.924251,96.001226,86.338404,4.985339,894.855094,1.429816,0.008329,0.000087,0.008295,0.000000,0.000000,0.899253,0.051925,0.054584,0.000000


In [20]:
df_cords = pd.read_csv('data/lakes_dashboard.csv')
df_cords.head()


,name,Unnamed: 0,impervious_fraction,max_3day_rain_mm,peak_30min_intensity_mm,sar_flood_freq_pct,potential_ha,static_water_ha,static_weed_ha,elevation,...,csr_ratio,log_flow,urban_stress,urban_surge,water_retention_ratio,weed_clogging_ratio,biological_clogging,rain_intensity_impact,lat,lon
0,Abbigere Lake,520.0,0.326674,81.433331,7.368333,16.895198,9.899677,1.206915,4.293971,883.016434,...,0.555661,1.872789,1.798803,2.321469,0.121792,0.433311,0.779180,2.321469,13.079432,77.528588
1,Agara Lake,464.0,0.023121,81.659998,7.086667,5.764079,26.724699,18.592974,4.461419,877.305438,...,0.000312,0.008301,0.000193,0.164046,0.695462,0.166877,0.193433,0.164046,12.920764,77.641364
2,Agrahara Lake,415.0,0.539475,84.547498,6.272500,13.790324,4.266708,2.033760,0.299592,899.184482,...,0.654447,1.334706,1.509925,3.296539,0.475543,0.070052,0.127848,3.296539,13.089487,77.624848
3,Akshaynagar Lake,455.0,0.945938,86.146664,6.710833,4.888746,1.187524,0.404341,0.467331,901.448762,...,0.132279,0.147046,0.149844,6.349636,0.337647,0.390248,0.530051,6.349636,12.871746,77.614269
4,Allalasandra Lake,535.0,0.184137,81.433331,7.368333,3.631073,13.948385,12.030289,1.809916,903.357495,...,0.000597,0.008295,0.001534,1.362220,0.861868,0.129665,0.130678,1.362220,13.091180,77.587238


In [21]:
cords = df_cords[['name', 'lat', 'lon']]
mean_df = mean_df.merge(cords, how='left', on='name')
mean_df.head()

,name,impervious_fraction,max_3day_rain_mm,peak_30min_intensity_mm,sar_flood_freq_pct,potential_ha,static_water_ha,static_weed_ha,elevation,slope,...,csr_ratio,log_flow,urban_stress,urban_surge,water_retention_ratio,weed_clogging_ratio,biological_clogging,rain_intensity_impact,lat,lon
0,Abbigere Lake,0.326674,81.433331,7.368333,16.895198,9.899677,1.206915,4.293971,883.016434,3.153620,...,0.555661,1.872789,1.798803,2.321469,0.121792,0.433311,0.779180,2.321469,13.079432,77.528588
1,Agara Lake,0.023121,81.659998,7.086667,5.764079,26.724699,18.592974,4.461419,877.305438,1.844719,...,0.000312,0.008301,0.000193,0.164046,0.695462,0.166877,0.193433,0.164046,12.920764,77.641364
2,Agrahara Lake,0.539475,84.547498,6.272500,13.790324,4.266708,2.033760,0.299592,899.184482,6.437909,...,0.654447,1.334706,1.509925,3.296539,0.475543,0.070052,0.127848,3.296539,13.089487,77.624848
3,Akshaynagar Lake,0.945938,86.146664,6.710833,4.888746,1.187524,0.404341,0.467331,901.448762,2.535673,...,0.132279,0.147046,0.149844,6.349636,0.337647,0.390248,0.530051,6.349636,12.871746,77.614269
4,Allalasandra Lake,0.184137,81.433331,7.368333,3.631073,13.948385,12.030289,1.809916,903.357495,1.944574,...,0.000597,0.008295,0.001534,1.362220,0.861868,0.129665,0.130678,1.362220,13.091180,77.587238


In [4]:
rain_feats = ['max_3day_rain_mm', 'peak_30min_intensity_mm'] 
urban_feats = ['impervious_fraction', 'urban_stress']
physical_feats = ['potential_ha', 'csr_ratio', 'elevation', 'slope', 'biological_clogging', 'log_flow']

In [5]:
X_cols = rain_feats + urban_feats + physical_feats
target_col = 'sar_flood_freq_pct'
df_ml = mean_df.dropna(subset=X_cols + [target_col])

X = df_ml[X_cols]
y = df_ml[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=1)
model.fit(X_train, y_train)

,"loss loss: {'squared_error', 'absolute_error', 'huber', 'quantile'}, default='squared_error'Loss function to be optimized. 'squared_error' refers to the squarederror for regression. 'absolute_error' refers to the absolute error ofregression and is a robust loss function. 'huber' is acombination of the two. 'quantile' allows quantile regression (use`alpha` to specify the quantile).See:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_quantile.py`for an example that demonstrates quantile regression for creatingprediction intervals with `loss='quantile'`.",'squared_error'
,"learning_rate learning_rate: float, default=0.1Learning rate shrinks the contribution of each tree by `learning_rate`.There is a trade-off between learning_rate and n_estimators.Values must be in the range `[0.0, inf)`.",0.05
,"n_estimators n_estimators: int, default=100The number of boosting stages to perform. Gradient boostingis fairly robust to over-fitting so a large number usuallyresults in better performance.Values must be in the range `[1, inf)`.",300
,"subsample subsample: float, default=1.0The fraction of samples to be used for fitting the individual baselearners. If smaller than 1.0 this results in Stochastic GradientBoosting. `subsample` interacts with the parameter `n_estimators`.Choosing `subsample < 1.0` leads to a reduction of varianceand an increase in bias.Values must be in the range `(0.0, 1.0]`.",1.0
,"criterion criterion: {'friedman_mse', 'squared_error'}, default='friedman_mse'The function to measure the quality of a split. Supported criteria are""friedman_mse"" for the mean squared error with improvement score byFriedman, ""squared_error"" for mean squared error. The default value of""friedman_mse"" is generally the best as it can provide a betterapproximation in some cases... versionadded:: 0.18",'friedman_mse'
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, values must be in the range `[2, inf)`.- If float, values must be in the range `(0.0, 1.0]` and `min_samples_split` will be `ceil(min_samples_split * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, values must be in the range `[1, inf)`.- If float, values must be in the range `(0.0, 1.0)` and `min_samples_leaf` will be `ceil(min_samples_leaf * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.Values must be in the range `[0.0, 0.5]`.",0.0
,"max_depth max_depth: int or None, default=3Maximum depth of the individual regression estimators. The maximumdepth limits the number of nodes in the tree. Tune this parameterfor best performance; the best value depends on the interactionof the input variables. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.If int, values must be in the range `[1, inf)`.",4
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.Values must be in the range `[0.0, inf)`.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft

In [6]:
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Regression Performance (R-squared): {r2:.4f}")
print(f"Mean Absolute Error: {mae:.2f}% flood frequency")

Regression Performance (R-squared): 0.6464
Mean Absolute Error: 3.74% flood frequency


In [7]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print(f"Cross-Validated R2: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

Cross-Validated R2: 0.5071 (+/- 0.0557)


In [8]:
importances = pd.Series(model.feature_importances_, index=X_cols).sort_values(ascending=False)
print("\nFeature Importance:")
print(importances)


Feature Importance:
biological_clogging        0.298668
impervious_fraction        0.220276
urban_stress               0.156373
potential_ha               0.099573
peak_30min_intensity_mm    0.067589
elevation                  0.052179
log_flow                   0.043120
slope                      0.024074
max_3day_rain_mm           0.022374
csr_ratio                  0.015774
dtype: float64


In [ ]:
df_ml[target_col]

name
Abbigere Lake                16.895198
Agara Lake                    5.764079
Agrahara Lake                13.790324
Akshaynagar Lake              4.888746
Allalasandra Lake             3.631073
                               ...    
Yediyur Lake                  4.153899
Yelachenahalli Kere           6.048937
Yelahanka Lake                0.924251
Yelenahalli Lake              6.172056
Yellamallappa Chetty Lake     0.000000
Name: sar_flood_freq_pct, Length: 162, dtype: float64

In [ ]:
lake_agg = df_ml[X_cols]
lake_agg['predicted_flood_freq'] = model.predict(lake_agg)
lake_agg['actual_flood_freq'] = df_ml[target_col]

lake_agg.head()

In [14]:
top_predicted = lake_agg.sort_values('predicted_flood_freq', ascending=False).head(15)
print("\nTop 15 Lakes by Predicted Flood Frequency (Refined Model):")
print(top_predicted[['predicted_flood_freq', 'actual_flood_freq']])


Top 15 Lakes by Predicted Flood Frequency (Refined Model):
                       predicted_flood_freq  actual_flood_freq
name                                                          
Doddabommasandra Kere             49.496680          49.628832
Attur Lake                        38.099688          31.986496
Gottigere Tank                    35.641134          30.975700
Bellandur Lake                    31.712188          31.725350
Horamavu Agara Lake               31.326218          31.439980
Hulimavu Lake                     31.088321          31.196252
Kogilu Lake                       29.179972          36.168563
Ramapura Kere                     28.886478          28.924602
Doddanekundi Lake                 28.210999          28.274660
Arekere lake                      27.436818          27.531883
Mallathahalli Lake                27.401938          27.457959
Hosakere                          24.704323          24.804512
Varthur Lake                      24.610366          24.55

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Adding multiple traces manually for precise control
fig.add_trace(go.Scatter(x=[1, 2, 3], y=[4, 5, 6], name="Actuals",
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x=[1, 2, 3], y=[2, 3, 4], name="Forecast",
                         line=dict(dash='dash')))

# Advanced Layout: Dual Axes
fig.update_layout(
    title_text="Professional Financial Forecast",
    template="plotly_white", # The "Clean" industry standard
    hovermode="x unified",    # Unified hover labels for comparison
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)



In [19]:
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        direction="left",
        buttons=list([
            dict(args=["type", "surface"], label="3D Surface", method="restyle"),
            dict(args=["type", "heatmap"], label="Heatmap", method="restyle")
        ]),
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.11,
        xanchor="left",
        y=1.1,
        yanchor="top"
    )]
)

In [18]:
import plotly.io as pio

# Create a custom corporate template
pio.templates["company_theme"] = go.layout.Template(
    layout=go.Layout(
        font=dict(family="Arial", size=12, color="#2c3e50"),
        paper_bgcolor="#f8f9fa",
        colorway=["#004a99", "#ee3124", "#ffc20e"] # Corporate palette
    )
)

# Apply globally
pio.templates.default = "company_theme"